# Experiment 1

This id is used as a prefix for the figure names.

In [ ]:
exp_id = 'experiment1'
from datetime import datetime
exp_id += '_' + str(datetime.now()).replace(' ', '_')

### Imports

In [ ]:
import sys
sys.path.insert(1, '../')

In [ ]:
from utils import plot_graph
from graph_loaders import load_graph
import matplotlib.pyplot as plt
import numpy as np
from approx import GBFGreedy
from kernels import VarSpline, Diffusion
import networkx as nx
plt.rcParams.update({'font.size': 16})

### Load a graph

In [ ]:
path = '../'
# G = load_graph('sensor2', path=path)
# G = load_graph('sensor1', path=path)
# G = load_graph('emptyset', path=path)
# G = load_graph('2moon', path=path)
# G = load_graph('minnesota', path=path)
# G = load_graph('rand', path=path)
# G = load_graph('rand_sparse', path=path)
G = load_graph('bunny', path=path)

# G = nx.dorogovtsev_goltsev_mendes_graph(7)
# pos = nx.spectral_layout(G, center=[0.5, 0.5])
# nx.set_node_attributes(G, pos, 'pos')

### Define an optimization set

In [ ]:
X_train = np.arange(len(G))
y_train = np.ones(len(G))

### Define a kernel

In [ ]:
# kernel = VarSpline(G, par=[-1.1, 0.01])
kernel = Diffusion(G, par=[-10])

### Select the points

In [ ]:
max_iter = 20 # Max number of point to be selected
tol_p = 1e-12  # Tolerance on the max of the squared power function
tol_f = 1e-12      # Tolerance on the residual

In [ ]:
model = GBFGreedy(G, kernel=kernel, greedy_type='p_greedy', 
                  reg_par=0, 
                  max_iter=max_iter, tol_p=tol_p, tol_f=tol_f,
                  verbose=False)

In [ ]:
model.fit(X_train, y_train)

### Visualize the selected points

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
plot_graph(G, ax=ax, values=model.eval_power_fun(X_train), 
           nodelist=model.ctrs_, 
           cb_label='Standard deviation')

plt.savefig('figures/' + exp_id + '_points' + '.pdf', bbox_inches='tight')

In [ ]:
model.eval_power_fun(X_train) / model.kernel.diagonal(X_train)

### Visualize the decay of the standard deviation

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
ax.plot(model.train_hist['p'])
ax.legend(['Max of the standard deviation'], fontsize=16)
ax.set_xlabel('Number of nodes', fontsize=16)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
ax.grid(True)

plt.savefig('figures/' + exp_id + '_p_max' + '.pdf', bbox_inches='tight')

### Visualize the decay of the residual

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
ax.plot(model.train_hist['f'])
ax.legend(['Max of the residual'], fontsize=16)
ax.set_xlabel('Number of nodes', fontsize=16)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
ax.grid(True)

plt.savefig('figures/' + exp_id + '_f_max' + '.pdf', bbox_inches='tight')